## Example of adaptation task without learning

Simulation of 10 trials with prism deviation = 25°, with a static model of cerebellum forward and inverse modules.

In [ ]:
client = get_hbp_service_client()
client.storage.download_file("/79916/scaffold_network_BSP.hdf5", "scaffold_network_BSP.hdf5")
client.storage.download_file("/79916/scaffold_network_BSP.json", "scaffold_network_BSP.json")
client.storage.download_file("/79916/scaffold_network_BSP_static.json", "scaffold_network_BSP_static.json")

import os
os.environ["NEST_MODULE_PATH"] = "/home/jovyan/nest-simulator-2.18.0-build/lib/nest"
os.environ["SLI_PATH"] = "/home/jovyan/nest-simulator-2.18.0-build/share/nest/sli"
os.environ["LD_LIBRARY_PATH"] = "/home/jovyan/nest-simulator-2.18.0-build/lib/nest:/home/jovyan/bin/lib"
os.environ["PATH"] = "/home/jovyan/bin/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin"
os.environ["SPATIALINDEX_C_LIBRARY"] = "/home/jovyan/bin/lib/libspatialindex.so"
os.environ["PYTHONPATH"] = "/home/jovyan/extra-cereb-nest/Tests:/opt/amber18/lib/python3.6/site-packages/:/home/jovyan/.local/nrn/lib/python:"
import sys
sys.path.insert(0, "/home/jovyan/nest-simulator-2.18.0-build/lib/python3.6/site-packages/")
sys.path.append("/home/jovyan/extra-cereb-nest/Tests")

import nest
nest.Install("cerebmodule")
nest.Install("extracerebmodule")

import numpy as np
import world
from population_view import PopView
from world_populations import Planner, Cortex, SensoryIO, MotorIO, DirectDCN, InverseDCN
from scaffold.core import from_hdf5
from scaffold.output import HDF5Formatter
from scaffold.config import JSONConfig

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.close_figures=False # keep figures open in pyplot
import auxiliary_functions as aux


trial_len = 300  # [ms]
n_trials = 10  # Number of trials for learning
CORES = 16  # We use all the 16 cores available
nest.set_verbosity("M_WARNING")
nest.ResetKernel()
nest.SetKernelStatus(
    {
        "local_num_threads": CORES,
        "total_num_virtual_procs": CORES,
        "resolution": 1.0,
        "overwrite_files": True,
    }
)

Create a brain with a static cerebellum

In [ ]:
# Reconfigure scaffold
static = True
filename_h5 = "scaffold_network_BSP.hdf5"
if static:
    filename_config = "scaffold_network_BSP_static.json"
else:
    filename_config = "scaffold_network_BSP.json"
reconfigured_obj = JSONConfig(filename_config)
HDF5Formatter.reconfigure(filename_h5, reconfigured_obj)

# Create scaffold_model from HDF5
scaffold_model = from_hdf5(filename_h5)
scaffold_model.configuration.verbosity = 1

# Get scaffold model populations
S_MF = scaffold_model.get_entities_by_type("mossy_fibers")
S_IO = scaffold_model.get_cells_by_type("io_cell")[:, 0]
S_DCN = scaffold_model.get_cells_by_type("dcn_cell")[:, 0]

uz_pos = scaffold_model.labels["microzone-positive"]
uz_neg = scaffold_model.labels["microzone-negative"]

S_DCNp = np.intersect1d(S_DCN, uz_pos)
S_DCNn = np.intersect1d(S_DCN, uz_neg)
S_IOp = np.intersect1d(S_IO, uz_pos)
S_IOn = np.intersect1d(S_IO, uz_neg)

# Prepare adapters
adapter_forward = scaffold_model.create_adapter("BSP_USECASE")
adapter_inverse = scaffold_model.create_adapter("BSP_USECASE")

adapter_forward.enable_multi("forward")
adapter_inverse.enable_multi("inverse")

adapter_forward.prepare()
adapter_inverse.prepare()

# Get NEST populations
f_IOp = adapter_forward.get_nest_ids(S_IOp)
f_IOn = adapter_forward.get_nest_ids(S_IOn)

i_IOp = adapter_inverse.get_nest_ids(S_IOp)
i_IOn = adapter_inverse.get_nest_ids(S_IOn)

f_DCNp = adapter_forward.get_nest_ids(S_DCNp)
f_DCNn = adapter_forward.get_nest_ids(S_DCNn)

i_DCNp = adapter_forward.get_nest_ids(S_DCNp)
i_DCNn = adapter_forward.get_nest_ids(S_DCNn)

f_MF = adapter_forward.get_nest_ids(S_MF)
i_MF = adapter_inverse.get_nest_ids(S_MF)

# Define population views
MF_number = len(f_MF)
IO_number = len(f_IOp) + len(f_IOn)

planner_pv = Planner(MF_number, prism=0.0, baseline_rate=10.0, gain_rate=2.0)
cortex_pv = Cortex(MF_number, rbf_sdev=MF_number/32, baseline_rate=200.0, gain_rate=5.0)

f_IO_pv = SensoryIO(f_IOp, f_IOn)  # External from the scaffold,
i_IO_pv = MotorIO(i_IOp, i_IOn)    # to be connected after

f_DCN_pv = DirectDCN(f_DCNp, f_DCNn)
i_DCN_pv = InverseDCN(i_DCNp, i_DCNn)

f_MF_pv = PopView(f_MF)
i_MF_pv = PopView(i_MF)


# Connections among different modules
conn_dict_one_to_one = {"rule": "one_to_one"}
nest.Connect(planner_pv.pop, cortex_pv.pop, conn_dict_one_to_one, {'weight': 1.0})

We retrieve the calibration values from the steps previous done

In [ ]:
# This is the calibration of the system
x_0 = 1.866743045267489  # From a previous calibration with the same network
x_10 = 3.765872592592592  # From a previous calibration with the same network
get_error = world.get_error_function(x_0, x_10)

Now we set a prism of 25° and we skip one trial and we compute the initial error.

In [ ]:
# Now the experiment can begin
planner_pv.set_prism(25)
trial_counter = 0
# Discard one trial
nest.Simulate(trial_len)
x = cortex_pv.integrate(trial_i=trial_counter)
trial_counter += 1

# Compute initial error
nest.Simulate(trial_len)
initial_error = get_error(cortex_pv.integrate(trial_i=trial_counter))
print("Calibrated Error: ", initial_error)
plt.close("all")
plt.figure()
plt.plot(get_error(cortex_pv.pos), "blue")
plt.ylabel("Position [deg]")
plt.xlabel("Time [ms]")
plt.ylim([-25, 35])
trial_counter += 1

Now connect the cerebellum scaffold modules

In [ ]:
nest.Connect(
    cortex_pv.pop, f_MF_pv.pop, conn_dict_one_to_one, {"weight": 1.0}
)  # Efference copy
nest.Connect(
    planner_pv.pop, i_MF_pv.pop, conn_dict_one_to_one, {"weight": 1.0}
)  # Sensory input

conn_dict_indegree = {"rule": "fixed_indegree", "indegree": 1}
nest.Connect(f_DCNp, cortex_pv.pop, conn_dict_indegree, {"weight": -2.0})
nest.Connect(f_DCNn, cortex_pv.pop, conn_dict_indegree, {"weight": 2.0})

Now we simulate 10 trials of the protocol

In [ ]:
sensory_error = initial_error
error_history = [initial_error]
cortex_history = [get_error(cortex_pv.pos)]
inverse_history = []
inversetor_history = []
plt.close("all")
plt.figure()
for i in range(n_trials):
    f_IO_pv.set_rate(sensory_error)
    i_IO_pv.set_rate(sensory_error, trial_i=trial_counter)
    nest.Simulate(trial_len)
    
    x_cortex = get_error(cortex_pv.integrate(trial_i=trial_counter))

    x_dcn = i_DCN_pv.integrate(trial_i=trial_counter)
    inverse_gain = -1.0
    sensory_error = x_cortex + inverse_gain * x_dcn
    error_history.append(sensory_error)
    print("Trial {}. Closed loop error: {}".format(i+1, sensory_error), end='\r')

    trial_counter += 1
    plt.clf()
    plt.plot(get_error(cortex_pv.pos), "blue")
    plt.plot(inverse_gain * i_DCN_pv.pos, "black")
    plt.plot(get_error(cortex_pv.pos)-inverse_gain * i_DCN_pv.pos, "red")
    plt.ylabel("Position [deg]")
    plt.xlabel("Time [ms]")
    plt.ylim([-25, 35])
    plt.legend(["motor cortex","cerebellum", "combined"])
    plt.title("Contribution to the final position (Desired is 0°)")
    plt.show()
    
    cortex_history.append(get_error(cortex_pv.pos))
    inverse_history.append(inverse_gain * i_DCN_pv.pos)
    inversetor_history.append(i_DCN_pv.torques)

##### Plot of the error along the 10 trials of adaptation
It is possible to see that no adaptation occurs, this is expected because we are using a static cerebellum

In [ ]:
np.savetxt("cortex_pos.dat", cortex_history)
np.savetxt("inverse_pos.dat", inverse_history)
np.savetxt("inverse_torques.dat", inversetor_history)
np.savetxt("static_error.dat", error_history[:-1])

plt.close("all")
plt.figure()
plt.plot(range(1, n_trials+1), error_history[:-1], "black")
plt.ylabel("Error [deg]")
plt.xlabel("Trial number")
plt.show()

Reset the Notebook kernel before continuing with a new simulation

In [ ]:
from IPython.display import display_html
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
    
restartkernel()

## Adaptation task with learning

Simulation of 10 trials with prism deviation = 25°, with plastic models of cerebellum forward and inverse modules.

In [ ]:
client = get_hbp_service_client()
client.storage.download_file("/79916/scaffold_network_BSP.hdf5", "scaffold_network_BSP.hdf5")
client.storage.download_file("/79916/scaffold_network_BSP.json", "scaffold_network_BSP.json")
client.storage.download_file("/79916/scaffold_network_BSP_static.json", "scaffold_network_BSP_static.json")

import os
os.environ["NEST_MODULE_PATH"] = "/home/jovyan/nest-simulator-2.18.0-build/lib/nest"
os.environ["SLI_PATH"] = "/home/jovyan/nest-simulator-2.18.0-build/share/nest/sli"
os.environ["LD_LIBRARY_PATH"] = "/home/jovyan/nest-simulator-2.18.0-build/lib/nest:/home/jovyan/bin/lib"
os.environ["PATH"] = "/home/jovyan/bin/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin"
os.environ["SPATIALINDEX_C_LIBRARY"] = "/home/jovyan/bin/lib/libspatialindex.so"
os.environ["PYTHONPATH"] = "/home/jovyan/extra-cereb-nest/Tests:/opt/amber18/lib/python3.6/site-packages/:/home/jovyan/.local/nrn/lib/python:"
import sys
sys.path.insert(0, "/home/jovyan/nest-simulator-2.18.0-build/lib/python3.6/site-packages/")
sys.path.append("/home/jovyan/extra-cereb-nest/Tests")

import nest
nest.Install("cerebmodule")
nest.Install("extracerebmodule")

import numpy as np
import world
from population_view import PopView
from world_populations import Planner, Cortex, SensoryIO, MotorIO, DirectDCN, InverseDCN
from scaffold.core import from_hdf5
from scaffold.output import HDF5Formatter
from scaffold.config import JSONConfig

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.close_figures=False # keep figures open in pyplot
import auxiliary_functions as aux


trial_len = 300  # [ms]
n_trials = 10  # Number of trials for learning
CORES = 16  # We use all the 16 cores available
nest.set_verbosity("M_WARNING")
nest.ResetKernel()
nest.SetKernelStatus(
    {
        "local_num_threads": CORES,
        "total_num_virtual_procs": CORES,
        "resolution": 1.0,
        "overwrite_files": True,
    }
)

Create a brain with a plastic cerebellum

In [ ]:
# Reconfigure scaffold
static = False
filename_h5 = "scaffold_network_BSP.hdf5"
if static:
    filename_config = "scaffold_network_BSP_static.json"
else:
    filename_config = "scaffold_network_BSP.json"
reconfigured_obj = JSONConfig(filename_config)
HDF5Formatter.reconfigure(filename_h5, reconfigured_obj)

# Create scaffold_model from HDF5
scaffold_model = from_hdf5(filename_h5)
scaffold_model.configuration.verbosity = 1

# Get scaffold model populations
S_MF = scaffold_model.get_entities_by_type("mossy_fibers")
S_IO = scaffold_model.get_cells_by_type("io_cell")[:, 0]
S_DCN = scaffold_model.get_cells_by_type("dcn_cell")[:, 0]

uz_pos = scaffold_model.labels["microzone-positive"]
uz_neg = scaffold_model.labels["microzone-negative"]

S_DCNp = np.intersect1d(S_DCN, uz_pos)
S_DCNn = np.intersect1d(S_DCN, uz_neg)
S_IOp = np.intersect1d(S_IO, uz_pos)
S_IOn = np.intersect1d(S_IO, uz_neg)

# Prepare adapters
adapter_forward = scaffold_model.create_adapter("FORWARD")
adapter_inverse = scaffold_model.create_adapter("INVERSE")

adapter_forward.enable_multi("forward")
adapter_inverse.enable_multi("inverse")

adapter_forward.prepare()
adapter_inverse.prepare()

# Get NEST populations
f_IOp = adapter_forward.get_nest_ids(S_IOp)
f_IOn = adapter_forward.get_nest_ids(S_IOn)

i_IOp = adapter_inverse.get_nest_ids(S_IOp)
i_IOn = adapter_inverse.get_nest_ids(S_IOn)

f_DCNp = adapter_forward.get_nest_ids(S_DCNp)
f_DCNn = adapter_forward.get_nest_ids(S_DCNn)

i_DCNp = adapter_forward.get_nest_ids(S_DCNp)
i_DCNn = adapter_forward.get_nest_ids(S_DCNn)

f_MF = adapter_forward.get_nest_ids(S_MF)
i_MF = adapter_inverse.get_nest_ids(S_MF)

# Define population views
MF_number = len(f_MF)
IO_number = len(f_IOp) + len(f_IOn)

planner_pv = Planner(MF_number, prism=0.0, baseline_rate=10.0, gain_rate=2.0)
cortex_pv = Cortex(MF_number, rbf_sdev=MF_number/32, baseline_rate=200.0, gain_rate=5.0)

f_IO_pv = SensoryIO(f_IOp, f_IOn)  # External from the scaffold,
i_IO_pv = MotorIO(i_IOp, i_IOn)    # to be connected after

f_DCN_pv = DirectDCN(f_DCNp, f_DCNn)
i_DCN_pv = InverseDCN(i_DCNp, i_DCNn)

f_MF_pv = PopView(f_MF)
i_MF_pv = PopView(i_MF)

# Connections among different modules
conn_dict_one_to_one = {"rule": "one_to_one"}
nest.Connect(planner_pv.pop, cortex_pv.pop, conn_dict_one_to_one, {'weight': 1.0})

We retrieve the calibration values from the steps previous done

In [ ]:
# This is the calibration of the system
x_0 = 1.866743045267489  # From a previous calibration with the same network
x_10 = 3.765872592592592  # From a previous calibration with the same network
get_error = world.get_error_function(x_0, x_10)

Now we set a prism of 25° and we skip one trial and we compute the initial error.

In [ ]:
# Now the experiment can begin
planner_pv.set_prism(25)
trial_counter = 0
# Discard one trial
nest.Simulate(trial_len)
x = cortex_pv.integrate(trial_i=trial_counter)
trial_counter += 1

# Compute initial error
nest.Simulate(trial_len)
initial_error = get_error(cortex_pv.integrate(trial_i=trial_counter))
print("Calibrated Error: ", initial_error)
plt.close("all")
plt.figure()
plt.plot(get_error(cortex_pv.pos), "blue")
plt.ylabel("Position [deg]")
plt.xlabel("Time [ms]")
plt.ylim([-25, 35])
trial_counter += 1

Now connect the cerebellum scaffold modules

In [ ]:
nest.Connect(
    cortex_pv.pop, f_MF_pv.pop, conn_dict_one_to_one, {"weight": 8.432319609339408828}
)  # Efference copy
nest.Connect(
    planner_pv.pop, i_MF_pv.pop, conn_dict_one_to_one, {"weight": 8.432319609339408828}
)  # Sensory input

conn_dict_indegree = {"rule": "fixed_indegree", "indegree": 1}
nest.Connect(f_DCNp, cortex_pv.pop, conn_dict_indegree, {"weight": -2.0})
nest.Connect(f_DCNn, cortex_pv.pop, conn_dict_indegree, {"weight": 2.0})

Now we simulate 10 trials of the protocol

In [ ]:
sensory_error = initial_error
error_history = [initial_error]
cortex_history = [get_error(cortex_pv.pos)]
inverse_history = []
inversetor_history = []
plt.close("all")
plt.figure()
for i in range(n_trials):
    f_IO_pv.set_rate(sensory_error)
    i_IO_pv.set_rate(sensory_error, trial_i=trial_counter)
    nest.Simulate(trial_len)
    
    x_cortex = get_error(cortex_pv.integrate(trial_i=trial_counter))

    x_dcn = i_DCN_pv.integrate(trial_i=trial_counter)
    inverse_gain = -1.207035362407005152e-02
    sensory_error = x_cortex + inverse_gain * x_dcn
    error_history.append(sensory_error)
    print("Trial {}. Closed loop error: {}".format(i+1, sensory_error), end='\r')

    trial_counter += 1
    plt.clf()
    plt.plot(get_error(cortex_pv.pos), "blue")
    plt.plot(inverse_gain * i_DCN_pv.pos, "black")
    plt.plot(get_error(cortex_pv.pos)-inverse_gain * i_DCN_pv.pos, "red")
    plt.ylabel("Position [deg]")
    plt.xlabel("Time [ms]")
    plt.ylim([-25, 35])
    plt.legend(["motor cortex","cerebellum", "combined"])
    plt.title("Contribution to the final position (Desired is 0°)")
    plt.show()
    
    cortex_history.append(get_error(cortex_pv.pos))
    inverse_history.append(inverse_gain * i_DCN_pv.pos)
    inversetor_history.append(i_DCN_pv.torques)

##### Plot of the error along the 10 trials of adaptation
It is possible to see that adaptation occurs and the error diminishes, thanks to the activation of both modules of the cerebellum.
See for example plots above where the contribution of both cerebellar models is visible:
- The inverse model generates a corrective command that decreases the final position (black line)
- The forward model modulates the contribution of the motor cortex (blue line), decreasing it as well

In [ ]:
np.savetxt("cortex_pos.dat", cortex_history)
np.savetxt("inverse_pos.dat", inverse_history)
np.savetxt("inverse_torques.dat", inversetor_history)
np.savetxt("plastic_error.dat", error_history[:-1])
static_error = np.loadtxt("static_error.dat")

plt.close("all")
plt.figure()
plt.plot(range(1, n_trials+1), error_history[:-1], "blue")
plt.plot(range(1, n_trials+1), static_error, "black")
plt.ylabel("Error [deg]")
plt.xlabel("Trial number")
plt.legend(["Plastic cerebellum", "Static cerebellum"])
plt.show()